# Тестовое задание:

требуется разработать модель, которая будет способна различать заголовки реальных и выдуманных новостей.

Для обучения модели используйте данные из файла `train.tsv`. В файле находится таблица, состоящая из двух колонок. 
В колонке title записан заголовок новости. В колонке is_fake содержатся метки: 0 – новость реальная; 1 – новость выдуманная.
Для демонстрации работы модели используйте данные тестового набора из файла `test.tsv`. В нем также есть колонка title, данные которой являются входными для вашей модели.

Вам нужно скопировать файл `test.tsv`, переименовать его в `predictions.tsv` и заполнить колонку is_fake значениями предсказаний вашей модели, аналогично `train.tsv`. 
Изначально колонка заполнена значением 0.

---

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = pd.read_csv('dataset/train.tsv', sep = '\t')
#train_data.style.hide_index()
train_data.head(10)

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1
5,Российским студентам запретят учиться за рубежом,1
6,Путин пишет книгу об истории Украины,1
7,Россияне обхитрили рост цен,0
8,Звезда «Ворониных» раскрыл подробности о своем...,0
9,Microsoft объявила дату выхода очков дополненн...,0


In [22]:
# разделим тренировочный датасет на train/test - нам же нужно будет как-то оценить работоспособность модели

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data['title'], train_data['is_fake'], 
                                                    test_size = 0.1, random_state = 42)
X_train[:10]

1448    Шестая часть Kingdom Hearts обойдет стороной P...
5113    Кировская филармония организовала концерт для ...
4092    ВГТРК выставит Киеву счёт за расходы, связанны...
1272    Москва лишена права проведения чемпионата мира...
589     «Уралкалий» объяснил действия Минска конкурент...
1084           Месси решил побить вечный рекорд Барселоны
2940    В России запретят продавать сигареты с 23:00 д...
881     В «Сбербанке Онлайн» появится функция платного...
5339    Женщин, уличённых в супружеской измене, будут ...
3576    Красный Крест требует, чтобы ЧВК «Вагнер» пере...
Name: title, dtype: object

# 1. подход TF-IDF:

In [23]:
# Токенизируем входные данные и строим словарь всех известных слов

from sklearn.svm import SVC 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

vectorizer = TfidfVectorizer()
train_x_vect = vectorizer.fit_transform(X_train)

tmp = train_x_vect.toarray().shape
print('по всем {0} сообщениям набралось {1} уникальных слов'.format(tmp[0], tmp[1]))

по всем 5182 сообщениям набралось 17531 уникальных слов


# 1.1. Support Vector Machine:

In [32]:
# Попробуем метод опорных векоторов с линейным ядром - выберем наиболее оптимальный параметр перебором

grid = {'C': np.power(10.0, np.arange(-5, 6))}

cv = KFold(n_splits = 5, shuffle = True, random_state = 42)
clf = SVC(kernel = 'linear', random_state = 42)

gs = GridSearchCV(clf, grid, scoring = 'f1', cv = cv)
gs.fit(train_x_vect, y_train)

print(gs.best_score_)
print(gs.best_params_)

0.8213058223375127
{'C': 1.0}


In [33]:
svc_clf = SVC(kernel = 'linear', random_state = 42, C = gs.best_params_['C'])
svc_clf.fit(train_x_vect, y_train)

SVC(kernel='linear', random_state=42)

In [34]:
#20 самых важных для моедли слов - можно также поработать над удалением стоп слов (что, чтобы, как итд)

A = svc_clf.coef_.indices
B = np.abs(svc_clf.coef_.data)

Words = A[B.argsort()[-20:][::-1]]
for i in Words:
    print(vectorizer.get_feature_names()[i])

запретят
навального
рпц
что
лукашенко
процентов
млн
как
россиян
нефти
qr
коронавируса
чтобы
учёные
сколково
шойгу
нефть
госдуме
2022
гибдд


In [35]:
#проверяем на тестовой выборке:

from sklearn.metrics import f1_score

y_pred = svc_clf.predict(vectorizer.transform(X_test))
print('f1_score = {}'.format(f1_score(y_test, y_pred, average = 'binary')))

f1_score = 0.8488745980707396


In [36]:
# Ответ на тестовых данных:

test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = svc_clf.predict(vectorizer.transform(test_data['title']))
#test_data.style.hide_index()
test_data.head(10)

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,1
9,Учреждена театральная премия имени Гарольда Пи...,0


Судя по поисковым запросам есть подозрение, что все эти новости на самом деле правдивые, а само задание просто шутка :)

..

либо это знак, что мне надо что-то предпринять для уменьшения ложноположительных ошибок.

# 1.2. Random Forest Classifier:

In [46]:
# Найдём оптимальные параметры перебором:

from sklearn.ensemble import RandomForestClassifier as RFC

grid = {'max_depth': np.arange(2, 20)}
cv = KFold(n_splits = 5, shuffle = True, random_state = 42)
clf = RFC(random_state = 42)

gs = GridSearchCV(clf, grid, scoring = 'f1', cv = cv)
gs.fit(train_x_vect, y_train)

print(gs.best_score_)
print(gs.best_params_)

0.7065052045215694
{'max_depth': 19}


In [47]:

rfc_clf = RandomForestClassifier(max_depth = gs.best_params_['max_depth'], random_state = 42)
rfc_clf.fit(train_x_vect, y_train)

# F1_score на тестовой выборке
y_pred = rfc_clf.predict(vectorizer.transform(X_test))
print('f1_score = {}'.format(f1_score(y_test, y_pred, average = 'binary')))

f1_score = 0.6505050505050505


In [48]:
# Ответ на тестовых данных:

test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = rfc_clf.predict(vectorizer.transform(test_data['title']))
#test_data.style.hide_index()
test_data.head(10)

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,1
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,1
9,Учреждена театральная премия имени Гарольда Пи...,0


# 2. другие подходы (?):

# 3.Вывод:

Наилучшую точность по метрике F1 показал подход TF_IDF  в сочетании с SVC

In [62]:
test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = svc_clf.predict(vectorizer.transform(test_data['title']))
test_data.to_csv('predictions.tsv', index = False, sep = '\t')